In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Read the file as html by BeautifulSoup and html parser 
with open("./500118.txt", encoding='utf-8') as fp:
    soup = BeautifulSoup(fp, "html.parser")

In [3]:
# Create data frame to add the data collected 
df = pd.DataFrame()

In [4]:
# features name
features = { 0:'color', 1:'fuel_type', 2:'car_origin',3:'driving_license',
                        4:'gear_type', 5:'glass', 6:'Motor_power', 7:'Vehicle_counter',
                        8:'passengers_numbers', 9:'Payment_method', 10:'Offered', 11:'Former_owners',
                        12:'need_split',13:'ad_status',14:'start_date',15:'end_date'}

In [5]:
#  Looking for the tr with the classes 'list-row'
table = soup.findAll('tr', class_='list-row')

# list for instance
data_row = []

# sting of each value
feature_value = ''

# Collecting Data
for row in table:  
    # Find all data for each column
    columns = row.findAll('td')
    addition_info = row.findAll('li')
    
    # Get the sample values as list
    if(columns != []):
        if(addition_info):
            for i in range(len(addition_info)):
                feature_value += addition_info[i].text + ','
        else:
            feature_value = columns[1].text
        
        data_row.append(feature_value)

# convert the data list to df
data_row = pd.DataFrame(data_row).rename(index=features).T

# add the final sample to initial df
df = df.append(data_row)

C:\Users\DELL\AppData\Local\Temp\ipykernel_20612\2835300177.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data_row)


In [6]:
# display df
df

,color,fuel_type,car_origin,driving_license,gear_type,glass,Motor_power,Vehicle_counter,passengers_numbers,Payment_method,Offered,Former_owners,need_split,ad_status,start_date,end_date
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,75000,4+1,نقدا فقط,للبيع فقط,يد اولى,"يد اولى مُكيّف,إغلاق مركزي,جهاز إنذار,مسجل CD,...",إعلان منتهي,2017-08-16,2017-09-15
